In [32]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from tqdm import tqdm

In [33]:
# Set random seed for reproducibility
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [34]:
# 1. Load and preprocess the data
def load_and_preprocess_data():
    # Assuming your data is in a CSV file
    df = pd.read_csv('/kaggle/input/amazon-electronices/amazon_reviews.csv')  
    
    # Convert overall ratings to binary sentiment
    # 4,5 = Positive (1), 1,2,3 = Negative (0)
    df['sentiment'] = df['overall'].apply(lambda x: 1 if x >= 4 else 0)
    
    # Handle missing reviewText
    df['reviewText'] = df['reviewText'].fillna('')
    
    return df[['reviewText', 'sentiment']]

In [35]:
# 2. Custom Dataset
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [37]:
# 3. Load data
df = load_and_preprocess_data()
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['reviewText'].values,
    df['sentiment'].values,
    test_size=0.2,
    random_state=42
)

# 4. Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 5. Dataset and DataLoader
train_dataset = ReviewDataset(train_texts, train_labels, tokenizer)
val_dataset = ReviewDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 6. Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# 7. Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# 8. Training loop
epochs = 10
train_loss_values, val_loss_values = [], []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f'Epoch {epoch+1}')
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())
    avg_train_loss = total_loss / len(train_loader)
    train_loss_values.append(avg_train_loss)

    # Validation
    model.eval()
    val_loss = 0
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten()
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_loss_values.append(avg_val_loss)

    acc = accuracy_score(true_labels, predictions)
    print(f'Validation Loss: {avg_val_loss:.4f} | Accuracy: {acc:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 100/100 [00:23<00:00,  4.23it/s, loss=0.254]


Validation Loss: 0.3127 | Accuracy: 0.8425


Epoch 2: 100%|██████████| 100/100 [00:23<00:00,  4.25it/s, loss=0.0711]


Validation Loss: 0.2933 | Accuracy: 0.8750


Epoch 3: 100%|██████████| 100/100 [00:23<00:00,  4.25it/s, loss=0.038] 


...

Validation Loss: 0.6345 | Accuracy: 0.8850


Epoch 9: 100%|██████████| 100/100 [00:23<00:00,  4.25it/s, loss=0.0019] 


Validation Loss: 0.5821 | Accuracy: 0.8850


Epoch 10: 100%|██████████| 100/100 [00:23<00:00,  4.25it/s, loss=0.00263]


Validation Loss: 0.6301 | Accuracy: 0.8700


In [38]:
# 9. Classification report
print("Classification Report:")
print(classification_report(true_labels, predictions))

# 10. Save model
model.save_pretrained('bert_sentiment_model')
tokenizer.save_pretrained('bert_sentiment_model')

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.66      0.68        82
           1       0.91      0.92      0.92       318

    accuracy                           0.87       400
   macro avg       0.80      0.79      0.80       400
weighted avg       0.87      0.87      0.87       400



('bert_sentiment_model/tokenizer_config.json',
 'bert_sentiment_model/special_tokens_map.json',
 'bert_sentiment_model/vocab.txt',
 'bert_sentiment_model/added_tokens.json')